# **<span style="color:red">Midterm 2 Solutions</span>**

## FINM 36700 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.


Submission
* You will upload your solution to the `Midterm 2` assignment on Canvas, where you downloaded this. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file has sheets for...
* `info` - names and descriptions of each factor
* `factors (excess returns)` - excess returns on several factors
* `portfolios (excess returns)` - excess returns on industry portfolios
* `risk-free rate` - risk-free rates over time

Note the data is **monthly** so any annualizations should use `12` months in a year.

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 30     |
| 2       | 35     |
| 3       | 20     |
| 4       | 15     |

### Each numbered question is worth 5 points unless otherwise specified.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\betamkt}{\beta^{i,\text{MKT}}}$$
$$\newcommand{\betahml}{\beta^{i,\text{HML}}}$$
$$\newcommand{\betaumd}{\beta^{i,\text{UMD}}}$$
$$\newcommand{\Eri}{E\left[\tilde{r}^{i}\right]}$$
$$\newcommand{\Emkt}{E\left[\tilde{r}^{\text{MKT}}\right]}$$
$$\newcommand{\Ehml}{E\left[\tilde{r}^{\text{HML}}\right]}$$
$$\newcommand{\Eumd}{E\left[\tilde{r}^{\text{UMD}}\right]}$$

# 1. Short Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

## 1.

Suppose that we find a set of factors that perfectly hedge any asset. Will these factors work as a linear factor pricing model? 

<font color='red'>

Yes. From the Lecture 5 slides, we have that under no arbitrage, if $\tilde{r}^p$ is perfectly replicated by the return factors, $x$, then by taking expectation of both sides we have a linear factor pricing model.

We can also argue this from the law of one price (LOOP). If $\tilde{r}^p$ is perfectly replicated by the return factors, $x$, then the LOOP implies that the price of the asset is the same as the price of the factors (with linear scaling), and therefore we have a pricing model.


</font>

## 2.

If the Fama-French 3-factor model fit perfectly, would the Treynor ratio be equal for every asset?

<font color='red'>
No. The key distinction here is that if we fit **only** the CAPM, then yes, as there is only the univariate Market beta. However, the 3-factor models has a **multivariate** market beta, which will not be the same as the one in the univariate case. Thus, even if the 3-factor model fits with a particular market premium, this will not be inferred from the Treynor ratio.
</font>

## 3.

Suppose the CAPM fits perfectly. Then assets which have higher time-series r-squared metrics on the market factor will have higher Sharpe ratios.

<font color='red'>
True. We saw that if the CAPM holds, every asset's Sharpe ratio equals the market Sharpe ratio scaled by correlation between the asset and the market. Thus, the higher the correlation, the higher the Sharpe ratio.

Note: r-squared being irrelevant to whether the pricing model fits is a seperate issue.

False is also a reasonable answer, **if and only if** the following reason is given:

$$
\textrm{SR}^i = (\rho_{i,\textrm{MKT}}) \cdot \textrm{SR}^{\textrm{MKT}} \;\;\;\; R^2 = \rho_{i,\textrm{MKT}}^2
$$
Note that this relationship only holds with the CAPM (since it is a univariate model). Then, since $R^2$ must be non-negative, but $\rho_{i,\textrm{MKT}}$ can be negative, then if an asset has a negative correlation with the market, then the Sharpe ratio will be negative, but the $R^2$ will be positive. Thus, the statement is false.

</font>

## 4.

Based on the case, what are two ways DFA hopes to generate attractive returns for investors?

<font color='red'>
Broad range of answers accepted.

- Taking advantage of factor premium, regardless of whether the factors are already known, or if they are rational or irrational.
- Minimizing transaction costs through block trades.
- Minimizing counterparty risk by not trading with investors who previously sold them "bad" small cap stocks.
- Tax optimization.

</font>

## 5.

We analyzed a strategy similar to "AQR's Momentum Funds" (mutual funds.) We found this implementation had much higher returns than the momentum factor of Fama French. What was a major drawback to this construction?

<font color='red'>
The fund had a much higher mean return and Sharpe ratio, but it was **long-only**. This caused it to have a correlation to the market of over 90%, and a high market beta. Thus, it had a significant amount of market risk, compared to long-short strategies which, in theory, have zero market beta.
</font>

## 6.

From our analysis, momentum has had a negative mean return since 2009. Is this evidence against momentum as a pricing factor? Explain why this is a problem or why it is not a problem.

<font color='red'>
The size of the momentum factor premium is not a challenge to a pricing model. You could fix this by just defining the factor in an opposite manner, for example: instead of a "value" factor, you could instead define a "growth" factor, which is just the opposite of the value factor. The mathematics would remain the same, but the interpretation would be different.

</font>

***

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import statsmodels.api as sm
import scipy.stats as stats
import arch
import warnings
warnings.filterwarnings("ignore")
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance
sns.set_theme()

def time_series_regression(portfolio, factors, FF3F = False, resid = False):
    
    ff_report = pd.DataFrame(index=portfolio.columns)
    bm_residuals = pd.DataFrame(columns=portfolio.columns)

    rhs = sm.add_constant(factors)

    for portf in portfolio.columns:
        lhs = portfolio[portf]
        res = sm.OLS(lhs, rhs, missing='drop').fit()
        ff_report.loc[portf, 'alpha_hat'] = res.params['const'] 
        ff_report.loc[portf, 'Market beta'] = res.params[1]
        if FF3F:
            ff_report.loc[portf, 'Value beta'] = res.params[2] 
            ff_report.loc[portf, 'Momentum beta'] = res.params[3]
            
        ff_report.loc[portf, 'info_ratio'] = np.sqrt(12) * res.params['const'] / res.resid.std()
        ff_report.loc[portf, 'treynor_ratio'] = 12 * portfolio[portf].mean() / res.params[1]
        ff_report.loc[portf, 'R-squared'] = res.rsquared
        ff_report.loc[portf, 'Tracking Error'] = (res.resid.std()*np.sqrt(12))

        if resid:
            bm_residuals[portf] = res.resid
            
            
        
    if resid:
        return bm_residuals
        
    return ff_report

factors = pd.read_excel('../data/midterm_2_data.xlsx', sheet_name = 'factors (excess returns)').set_index('Date')
portfolio = pd.read_excel('../data/midterm_2_data.xlsx', sheet_name = 'portfolios (excess returns)').set_index('Date')

# 2. Linear Factor Pricing Models (LFPMs)

This problem tests the following LFPM:

$$\begin{align}
\Eri = \betamkt \Emkt + \betahml \Ehml + \betaumd \Eumd
\end{align}$$

## 1.

### (8 pts)

Estimate the **time-series (TS)** test of this pricing model. 

For each asset, report the following statistics:
* annualized alpha
* betas
* r-squared

In [3]:
ts_FF = time_series_regression(portfolio, factors,True)
ts_FF['alpha_hat']*=12
ts_FF[['alpha_hat','Market beta', 'Value beta','Momentum beta','R-squared']]

,alpha_hat,Market beta,Value beta,Momentum beta,R-squared
NoDur,0.029253,0.739522,0.204580,0.049333,0.617919
Durbl,0.010734,1.271865,0.173595,-0.320023,0.613493
Manuf,-0.000996,1.049482,0.197462,-0.036704,0.870268
Enrgy,-0.015117,0.992222,0.637006,0.075170,0.465602
HiTec,0.028207,1.154959,-0.637135,-0.140638,0.829498
Telcm,0.003506,0.837326,0.094363,-0.084518,0.588052
Shops,0.026739,0.946928,-0.042222,-0.015005,0.742161
Hlth,0.031862,0.757605,-0.119928,0.074058,0.580514
Utils,0.013710,0.527879,0.353033,0.108622,0.342654
Other,-0.019780,1.115433,0.426753,-0.048678,0.910098


## 2.

### (7pts)

Estimate the **cross-sectional (CS)** test of the pricing model. 

Include an intercept in your cross-sectional test.

Report the
* annualized intercept
* annualized regression coefficients
* r-squared

In [8]:
mean_portfolios = (portfolio.mean()*12).to_frame('Mean Portfolio excess returns')
CS = time_series_regression(mean_portfolios, ts_FF.loc[:,['Market beta', 'Value beta','Momentum beta']], True)
CS.columns = ['Annualized Intercept','Market regression coefficient','Value regression coefficients',
            'Momentum regression coefficients','info_ratio','treynor_ratio','R-squared','Tracking Error']

display(CS[['Market regression coefficient','Value regression coefficients',
    'Momentum regression coefficients']].T)
display(CS[['Annualized Intercept','Market regression coefficient','Value regression coefficients',
    'Momentum regression coefficients','R-squared']].T)

,Mean Portfolio excess returns
Market regression coefficient,0.031992
Value regression coefficients,-0.015767
Momentum regression coefficients,0.030301


,Mean Portfolio excess returns
Annualized Intercept,0.063716
Market regression coefficient,0.031992
Value regression coefficients,-0.015767
Momentum regression coefficients,0.030301
R-squared,0.366198


## 3.

Report the annualized factor premia (expected excess returns of the three factors) as implied by each of the TS and CS estimations.

In [9]:
factor_premia = pd.DataFrame(index = ['TS','CS'], columns = ['MKT Premia','HML Premia','UMD Premia'])
factor_premia.loc['TS',:] = (factors.mean()*12).values
factor_premia.loc['CS',:] = CS[['Market regression coefficient','Value regression coefficients',
    'Momentum regression coefficients']].values           
factor_premia

,MKT Premia,HML Premia,UMD Premia
TS,0.083853,0.025028,0.061692
CS,0.031992,-0.015767,0.030301


## 4.

Use the r-squared statistics from the TS and CS tests above to assess whether these factors are effective for decomposition and/or pricing.

Be specific as to how the r-squared statistics from the TS and CS tests impact your conclusions.

<font color='red'>

Nothing could be said about the R-Squared for the time-series case as in the TimeSeries test we do not care about high R-Squared. Thus, the average R-Squared statistic would be unrestricted. For the Cross-sectional test, we want ideally the R-squared to be 1.0, as we expect the factors to explain all of the expected returns of the portfolios. Since the r-squared in CS is not 1.0, there are some other factors that need to be included to perfectly explain the returns for all the assets. 

</font>

## 5.

Report the annualized pricing mean absolute error (MAE) implied by each of the TS and CS estimations.

In [10]:
resid = time_series_regression(mean_portfolios, ts_FF.loc[:,['Market beta', 'Value beta','Momentum beta']], False, True)
mae = pd.DataFrame([(((ts_FF['alpha_hat']).abs().mean()))], columns = ['MAE (%)'], index = ['TS'])
mae.loc['CS'] = abs(resid).mean()[0]
mae

,MAE (%)
TS,0.017990
CS,0.007915


## 6.

Which asset has the highest premium as implied by the TS estimation? And as implied by the CS estimation? (For the latter, feel free to include the cross-sectional intercept.)

In [11]:
ts = ts_FF.copy()
ts = ts.rename(columns={'Market beta':'MKT','Value beta':'HML','Momentum beta':'UMD'})
((factors.mean()*12)[['MKT','HML','UMD']] * ts[['MKT','HML','UMD']]).sum(axis =1).to_frame('TS Predicted Premium').nlargest(1, 'TS Predicted Premium')

,TS Predicted Premium
Enrgy,0.103782


In [12]:
cs = CS.copy()
cs = cs.rename(columns= {'Market regression coefficient':'MKT','Value regression coefficients':'HML',
                        'Momentum regression coefficients':'UMD'})
(ts[['MKT','HML','UMD']] * cs[['MKT','HML','UMD']].values[0]).sum(axis=1).nlargest(1).to_frame('CS Predicted Premium without intercept')

,CS Predicted Premium without intercept
HiTec,0.042734


In [13]:
predicted_cs = cs['Annualized Intercept'][0] + (ts[['MKT','HML','UMD']] * cs[['MKT','HML','UMD']].values[0]).sum(axis=1)
(predicted_cs).nlargest(1).to_frame('CS Predicted Premium with intercept')

,CS Predicted Premium with intercept
HiTec,0.10645


***

# 3. Additional Analysis

## 1. 

Consider the three-factor pricing model above. How can we assess whether all three factors are useful in this pricing model? 

Specifically, discuss whether the previously estimated regression betas would be informative. If not, what other statistic could we calculate?

<font color='red'>
We, unfortunately are unable to determine this by looking at the betas alone. That is, the betas are not informative in terms of usefulness of a specific factor; but rather when taken together they are informative in terms of the usefulness of the **entire** model.

The way to estimate if a given factor is useful would be to construct the tangency portfolio of the factors, and then look at the individual weights. Then, the factor with the largest weight would be the most useful, and the factor with the smallest weight would be the least useful.
</font>

## 2.

Suppose we are testing the 3-factor model above, and now we want to allow for time-varying betas.

How could we test the model while allowing for this?

Be specific about the number of regressions we would run and the nature of these regressions.

<font color='red'>
This comes from Fama-MacBeth at the end of Lecture 4. 

1. Estimate a the time-series regression using a rolling window or other methods, we want an estimate of $\beta_{t}^i$, that is, the beta for asset $i$ at time $t$.
2. Run a cross-sectional regression for each time, $t$. That is, calculate the cross-section regression using the $\beta_{t}$, for each time $t$. We do this to calculate $\lambda$ and $v$.
3. We can then take the sample means of each of the estimates to get the final estimates of $\lambda$ and $v$.

Let's assume we are using t-window rolling regression. For TS regression we have N * (T - t+1) regressions where N is the number of assets and T is the total number of time periods. For CS regression we have (T-t+1) regression for each of the time periods. Thus, the total number of regressions is (N + 1) * (T - t+1)

</font>

## 3.

State one advantage and one disadvantage of using the CS estimation as opposed to the TS estimation in fitting the LFPM to the data.

<font color='red'>
CS estimation has the advantage of fitting the model better, as we are able to adjust the factor premia (factor premia is now a free parameter).

The disadvantage is that the factor premia are estimated in CS, meaning that the estimated factor premia could be completely different to the factor premia estimated using historical data.

</font>

## 4.

Suppose we are investing in just the assets included in our data set. We want to implement a momentum strategy.

Relative to the momentum strategies we studied, do you expect this strategy would have higher or lower...
* mean
* volatility

Explain.

<font color='red'>
We only have 10 assets! Therefore, we would expect to have a much, much lower mean, as statistically speaking, there won't be the big winners and big losers that momentum relies on. That is, if our universe was 1000 securities, we'd expect to have much more extreme winners and losers in the tails than if we only had 10. Since momentum relies on these big winners and losers, then we would expect to have a much lower mean. We need extremes.

Either higher or lower vol is correct.

Higher vol: less diversification, since we are only investing in 10 assets.

Lower vol: the 10 assets are themselves portfolios, meaning that we are already diversified!

Overall, this strategy is a bad approach; we need to take extremes, and 10 assets make that very difficult.

</font>

***

# 4. Returns Over Time

## 1.

If Barnstable’s assumptions hold, (log iid returns, normally distributed,) then in what sense is an investment safer in the long-run? And in what sense is it riskier in the long-run?

<font color='red'>

Sharpe ratio scales with time, meaning that in the long-run we would expect to have a higher Sharpe, and thus it would be a safer investment. Additionally, we'll have lower standard error of average returns.

Riskier because volatility also scales with time! That means our cumulative returns will have higher volatility (despite a higher Sharpe) and thus they could be considered riskier in the long run. We also have parameter uncertainty, meaning that we are less certain about the parameters in the long run, so we have "model risk" in the long run.

</font>

## 2. 

### (10pts)

Data 
* Make use of the `risk-free rate` tab.
* Construct the **total** factor returns by adding the risk-free rate to the excess `MKT` and `HML` factor returns.

Assumptions
* The total returns are lognormally distributed and iid. 

Report the probability that `MKT` will outperform `HML` over the following 5 years.

<font color='red'>
First, we assume log-returns of MKT and HML are iid multivariate normal.

We want

$$P\left(\Pi_{t=1}^{60} (1+r_t^{MKT}) > \Pi_{t=1}^{60} (1+r_t^{HML})\right)$$

Denote log-return $\mathrm{r} = \log(1+r)$ and let $\mathrm{r}^P = \mathrm{r}^{MKT} - \mathrm{r}^{HML}$.



$$
\begin{aligned}
P\left(\Pi_{t=1}^{60} (1+r_t^{MKT}) > \Pi_{t=1}^{60} (1+r_t^{HML})\right) 
&= P\left(\sum_{t=1}^{60} \mathrm{r}_t^{MKT} > \sum_{t=1}^{60} \mathrm{r}_t^{HML}\right) \\
&= P\left(\sum_{t=1}^{60} (\mathrm{r}_t^{MKT} - \mathrm{r}_t^{HML}) > 0 \right) \\
&= P\left(\sum_{t=1}^{60} \mathrm{r}_t^{P} > 0 \right) \\
&= P\left(\bar{\mathrm{r}}_t^{P} > 0 \right) \\
&= 1- \Phi\left(\sqrt{60}\frac{-\mu}{\sigma}\right) = \Phi\left(\sqrt{60}\frac{\mu}{\sigma}\right)
\end{aligned}
$$

The last equation comes from

$$\bar{\mathrm{r}}^P \sim \mathcal{N}\left(\mu, \frac{1}{60}\sigma^2\right)$$

where

$$\mu = \mu_{MKT} - \mu_{HML}$$ 
$$\sigma^2 = \sigma_{MKT}^2 + \sigma_{HML}^2 - 2\sigma_{MKT, HML}$$

or can directly calculate $\mu$ and $\sigma$ from the spread between the two.

In case of using annualized $\mu$ and $\sigma$, the distribution should be the average of 5 years of annualized returns
$$\bar{\mathrm{r}}_{annual}^P \sim \mathcal{N}\left(12\mu, \frac{12}{5}\sigma^2\right)$$


$$
\Phi\left(\frac{12\mu}{\sqrt{12/5}\sigma}\right) = \Phi\left(\sqrt{60}\frac{\mu}{\sigma}\right)
$$

Which is equivalent to the above.

</font>

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm

data = pd.read_excel("../data/midterm_2_data.xlsx", sheet_name=None)

sheets = list(data.keys())
factors = data[sheets[1]].set_index('Date')
ports = data[sheets[2]].set_index('Date')
rf = data[sheets[3]].set_index('Date')


# Method one
total_log_ret = np.log(1 + factors.apply(lambda x: x +rf['RF']))

spread = total_log_ret['MKT'] - total_log_ret['HML']
mu, sigma = spread.mean(), spread.std()

prob = 1 - norm.cdf(0, mu, sigma/np.sqrt(60))
prob.round(2)

0.71

In [3]:
# Method two
spread = total_log_ret['MKT'] - total_log_ret['HML']
mu, sigma = spread.mean() * 12, spread.std() * np.sqrt(12)

prob = 1 - norm.cdf(0, mu, sigma/np.sqrt(5))
prob.round(2)

0.71